In [1]:
import re  
import nltk
import string
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

pd.set_option("display.max_colwidth",200)
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
train =pd.read_excel('Data_train.xlsx')
train.head()

,STORY,SECTION
0,"But the most painful was the huge reversal in fee income, unheard of among private sector lenders. Essentially, it means that Yes Bank took it for granted that fees on structured loan deals will b...",3
1,"How formidable is the opposition alliance among Congress, Jharkhand Mukti Morcha (JMM) and Jharkhand Vikas Morcha (Prajatantrik)?",0
2,"Most Asian currencies were trading lower today. South Korean won was down 0.4%, China renminbi 0.23%, China Offshore 0.15%, Malaysian ringgit 0.12%, Indonesian rupiah 0.11%, Taiwan dollar 0.06%. H...",3
3,"If you want to answer any question, click on ‘Answer’. After clicking on Answer you can also check out replies of other users. Proceed to Answer either through writing or voice command.\n\n\nIf yo...",1
4,"In global markets, gold prices edged up today as disappointing Chinese factory activity data brought back concerns about the health of the global economy, denting risk appetite. Spot gold rose 0.4...",3


In [3]:
test =pd.read_excel('Data_test.xlsx')
test.head()

,STORY
0,2019 will see gadgets like gaming smartphones and wearable medical devices lifting the user experience to a whole new level\n\n\nmint-india-wire consumer technologyconsumer technology trends in Ne...
1,It has also unleashed a wave of changes in the MCU that will make sure its future is a lot different than its past\n\n\nKevin Feige had signalled diversity and more representation in the post-phas...
2,"It can be confusing to pick the right smartphone for yourself, so we have segregated the top smartphones under Rs 20,000 according to their strengths.\n\n\nThe best smartphones under ₹20,000 categ..."
3,"The mobile application is integrated with a dashboard to confirm and register the pre-registered cases, to enable online interface between the beneficiary and the panel lawyer through video confer..."
4,"We have rounded up some of the gadgets that showed up in 2018 and left an indelible mark on, consumers, experts and the tech industry\n\n\nYoungsters playing PUBG Mobile on their smartphone for ho..."


In [4]:
train.shape , test.shape

((7628, 2), (2748, 1))

In [5]:
train.duplicated().sum(), test.duplicated().sum()

(77, 11)

In [5]:
train=train[train.duplicated()==False].reset_index()
train.shape

(7551, 3)

In [7]:
train.SECTION.value_counts()

1    2731
2    1914
0    1673
3    1233
Name: SECTION, dtype: int64

In [8]:
train.isna().sum(),test.isna().sum()

(index      0
 STORY      0
 SECTION    0
 dtype: int64, STORY    0
 dtype: int64)

In [6]:
combi = train.append(test , ignore_index=True)

C:\Users\Public\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [7]:
combi.STORY = combi.STORY.str.replace(r"\n+", " ")
combi.STORY = combi.STORY.str.replace(r"(\d)", "")
#combi.STORY = combi.STORY.str.replace(r".%", "")
combi.STORY = combi.STORY.str.replace(r"[^A-Za-z]", " ")
combi.STORY = combi.STORY.str.lower()

In [9]:
from nltk.corpus import stopwords

In [10]:
stop = stopwords.words('english')
combi.STORY= combi.STORY.apply(lambda x : " ".join([w for w in x.split() if w not in stop]))
combi.head()

,SECTION,STORY,index
0,3.0,painful huge reversal fee income unheard among private sector lenders essentially means yes bank took granted fees structured loan deals paid accounted upfront books borrowers turned defaulters fe...,0.0
1,0.0,formidable opposition alliance among congress jharkhand mukti morcha jmm jharkhand vikas morcha prajatantrik,1.0
2,3.0,asian currencies trading lower today south korean china renminbi china offshore malaysian ringgit indonesian rupiah taiwan dollar however japanese yen dollar index measures us currency strength ma...,2.0
3,1.0,want answer question click answer clicking answer also check replies users proceed answer either writing voice command want ask question click ask question question prefixes already inserted help ...,3.0
4,3.0,global markets gold prices edged today disappointing chinese factory activity data brought back concerns health global economy denting risk appetite spot gold rose per ounce european equity market...,4.0


In [11]:
tokenized_text = combi['STORY'].apply(lambda x: x.split())
tokenized_text.head()

0    [painful, huge, reversal, fee, income, unheard, among, private, sector, lenders, essentially, means, yes, bank, took, granted, fees, structured, loan, deals, paid, accounted, upfront, books, borro...
1                                                                                 [formidable, opposition, alliance, among, congress, jharkhand, mukti, morcha, jmm, jharkhand, vikas, morcha, prajatantrik]
2    [asian, currencies, trading, lower, today, south, korean, china, renminbi, china, offshore, malaysian, ringgit, indonesian, rupiah, taiwan, dollar, however, japanese, yen, dollar, index, measures,...
3    [want, answer, question, click, answer, clicking, answer, also, check, replies, users, proceed, answer, either, writing, voice, command, want, ask, question, click, ask, question, question, prefix...
4    [global, markets, gold, prices, edged, today, disappointing, chinese, factory, activity, data, brought, back, concerns, health, global, economy, denting, risk, appetite, spot,

In [15]:
#Stemming
from nltk.stem.porter import *
stemmer=PorterStemmer()
tokenized_text = tokenized_text.apply(lambda x: [stemmer.stem(i) for i in x])   #stemming
#Lemmatizing - is giving less accurate so used
#from nltk.stem import WordNetLemmatizer 
#lemmatizer = WordNetLemmatizer() 
#tokenized_text1 = tokenized_text.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])   #lemmatizer

In [16]:
#stitch these tokens back together
for i in range(len(tokenized_text)):
    tokenized_text[i] = ' '.join(tokenized_text[i])
combi['TIDY_STORY'] = tokenized_text

In [20]:
#Count Vectoriser
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer=CountVectorizer(max_df=0.90,min_df=2,max_features=1000,stop_words='english')
bow = bow_vectorizer.fit_transform(combi['TIDY_STORY'])
print(bow.shape)

(10299, 1000)


In [21]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(max_df=0.90,min_df=2,max_features=1000,stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(combi['TIDY_STORY'])
print(tfidf.shape)

(10299, 1000)


In [17]:
import gensim
tokenized_text = combi['TIDY_STORY'].apply(lambda x: x.split()) # tokenizing 

model_w2v = gensim.models.Word2Vec(
            tokenized_text,
            size=200, # desired no. of features/independent variables-300
            window=5, # context window size -7
            min_count=2,  # Minimum word count threshold
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 4, # no.of cores
            #sampling=#0=1e-3      #Downsample setting for frequent words-  1e-5  
            seed = 34) 
model_w2v.train(tokenized_text, total_examples= len(combi['TIDY_STORY']), epochs=20)

(12481057, 12900340)

In [18]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary                                     
            continue
    if count != 0:
        vec /= count
    return vec

In [19]:
wordvec_arrays = np.zeros((len(tokenized_text), 200)) 
for i in range(len(tokenized_text)):
    wordvec_arrays[i,:] = word_vector(tokenized_text[i], 200)
wordvec_df = pd.DataFrame(wordvec_arrays) 
print(wordvec_df.shape)   

(10299, 200)


In [192]:
from tqdm import tqdm 
tqdm.pandas(desc="progress-bar") 
from gensim.models.doc2vec import LabeledSentence

In [193]:
def add_label(twt):
    output = []
    for i, s in zip(twt.index, twt):
        output.append(LabeledSentence(s, ["text_" + str(i)]))
    return output
labeled_texts = add_label(tokenized_text) # label all the tweets

In [194]:
#Now let’s train a doc2vec model.
model_d2v = gensim.models.Doc2Vec(
    dm=1, # dm = 1 for ‘distributed memory’ model                                  
    dm_mean=1, # dm = 1 for using mean of the context word vectors                                 
    vector_size=200, # no. of desired features                                  
    window=5, # width of the context window                                  
    negative=7, # if > 0 then negative sampling will be used                                 
    min_count=5, # Ignores all words with total frequency lower than 2.                                  
    workers=3, # no. of cores                                  
    alpha=0.1, # learning rate                                  
    seed = 23) 
model_d2v.build_vocab([i for i in tqdm(labeled_texts)])
model_d2v.train(labeled_texts, total_examples= len(combi['TIDY_STORY']), epochs=15)

100%|████████████████████████████████| 10299/10299 [00:00<00:00, 936401.48it/s]


In [195]:
#Preparing doc2vec Feature Set
docvec_arrays = np.zeros((len(tokenized_text), 200)) 
for i in range(len(combi)):
    docvec_arrays[i,:] = model_d2v.docvecs[i].reshape((1,200))    

docvec_df = pd.DataFrame(docvec_arrays) 
print(docvec_df.shape)

(10299, 200)


In [22]:
# Splitting into  train and test features 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import f1_score , accuracy_score,confusion_matrix , classification_report

# Bag-of-Words Features
train_bow = bow[:7551,:]
test_bow = bow[7551:,:]
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow, train['SECTION'], random_state=1,test_size=0.2) #2

#TF_IDF Bag-of-Words Features
train_tfidf = tfidf[:7551,:]
test_tfidf = tfidf[7551:,:]
xtrain_tfidf = train_tfidf[ytrain.index] 
xvalid_tfidf = train_tfidf[yvalid.index]

#Word2Vec Features
train_w2v = wordvec_df.iloc[:7551,:]
test_w2v = wordvec_df.iloc[7551:,:] 
xtrain_w2v = train_w2v.iloc[ytrain.index,:] 
xvalid_w2v = train_w2v.iloc[yvalid.index,:]

#Doc2Vec Features
train_d2v = docvec_df.iloc[:7551,:] 
test_d2v = docvec_df.iloc[7551:,:] 
xtrain_d2v = train_d2v.iloc[ytrain.index,:] 
xvalid_d2v = train_d2v.iloc[yvalid.index,:]

NameError: name 'docvec_df' is not defined

In [157]:
#Bag-of-Words Features
from sklearn.model_selection import train_test_split 

from xgboost import XGBClassifier

xgb_model = XGBClassifier(max_depth=6, n_estimators=1000).fit(xtrain_bow, ytrain) 
prediction = xgb_model.predict(xvalid_bow) 
print('accuracy',accuracy_score(yvalid, prediction))  
print('f1score',f1_score(y_true=yvalid, y_pred=prediction, average='weighted'))
print(confusion_matrix(yvalid, prediction))  
print(classification_report(y_true=yvalid, y_pred=prediction))

accuracy 0.9510258107213766
f1score 0.9510284519624431
[[326   7  10   1]
 [  7 518  14  10]
 [  3   5 368   1]
 [  4   7   5 225]]
              precision    recall  f1-score   support

           0       0.96      0.95      0.95       344
           1       0.96      0.94      0.95       549
           2       0.93      0.98      0.95       377
           3       0.95      0.93      0.94       241

   micro avg       0.95      0.95      0.95      1511
   macro avg       0.95      0.95      0.95      1511
weighted avg       0.95      0.95      0.95      1511



In [184]:
xgb_model = XGBClassifier(max_depth=6, n_estimators=1000).fit(train_bow, train['SECTION']) 
test_pred = xgb_model.predict(test_bow) 
test['SECTION'] = test_pred 
submission = test['SECTION'] 
submission.to_excel('xgb_bow.xlsx', index=False)   #96.10

In [158]:
#TF-IDF Features
xgb = XGBClassifier(max_depth=6, n_estimators=1000).fit(xtrain_tfidf, ytrain) 
prediction = xgb.predict(xvalid_tfidf) 
print('accuracy',accuracy_score(yvalid, prediction))  
print('f1score',f1_score(y_true=yvalid, y_pred=prediction, average='weighted'))
print(confusion_matrix(yvalid, prediction))  
print(classification_report(y_true=yvalid, y_pred=prediction))

accuracy 0.9523494374586366
f1score 0.9524388622657395
[[326   5  12   1]
 [  4 520  16   9]
 [  2   6 367   2]
 [  3   7   5 226]]
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       344
           1       0.97      0.95      0.96       549
           2       0.92      0.97      0.94       377
           3       0.95      0.94      0.94       241

   micro avg       0.95      0.95      0.95      1511
   macro avg       0.95      0.95      0.95      1511
weighted avg       0.95      0.95      0.95      1511



In [185]:
xgb = XGBClassifier(max_depth=6, n_estimators=1000).fit(train_tfidf, train['SECTION']) 
test_pred = xgb.predict(test_bow) 
test['SECTION'] = test_pred 
submission = test['SECTION'] 
submission.to_excel('xgb_tfidf.xlsx', index=False)  #87.48

In [197]:
xgb = XGBClassifier(max_depth=6, n_estimators=1000, nthread= 3).fit(xtrain_w2v, ytrain) 
prediction = xgb.predict(xvalid_w2v) 
print('accuracy',accuracy_score(yvalid, prediction))  
print('f1score',f1_score(y_true=yvalid, y_pred=prediction, average='weighted'))
print(confusion_matrix(yvalid, prediction))  
print(classification_report(y_true=yvalid, y_pred=prediction))


accuracy 0.970880211780278
f1score 0.9708629935924304
[[328   8   6   2]
 [  2 534   6   7]
 [  5   2 370   0]
 [  3   2   1 235]]
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       344
           1       0.98      0.97      0.98       549
           2       0.97      0.98      0.97       377
           3       0.96      0.98      0.97       241

   micro avg       0.97      0.97      0.97      1511
   macro avg       0.97      0.97      0.97      1511
weighted avg       0.97      0.97      0.97      1511



In [198]:
xgb = XGBClassifier(max_depth=6, n_estimators=1000).fit(train_w2v, train['SECTION']) 
test_pred = xgb.predict(test_w2v) 
test['SECTION'] = test_pred 
submission = test['SECTION']   
submission.to_excel('xgb_w2v.xlsx', index=False)  #97.34

In [199]:
#Doc2Vec Features
xgb = XGBClassifier(max_depth=6, n_estimators=1000, nthread= 3).fit(xtrain_d2v, ytrain) 
prediction = xgb.predict(xvalid_d2v) 
print('accuracy',accuracy_score(yvalid, prediction))  
print('f1score',f1_score(y_true=yvalid, y_pred=prediction, average='weighted')) 
print(confusion_matrix(yvalid, prediction))  
print(classification_report(y_true=yvalid, y_pred=prediction))


accuracy 0.9126406353408338
f1score 0.9125662466644285
[[302  21  17   4]
 [ 11 506  19  13]
 [  7  16 351   3]
 [  6  10   5 220]]
              precision    recall  f1-score   support

           0       0.93      0.88      0.90       344
           1       0.92      0.92      0.92       549
           2       0.90      0.93      0.91       377
           3       0.92      0.91      0.91       241

   micro avg       0.91      0.91      0.91      1511
   macro avg       0.91      0.91      0.91      1511
weighted avg       0.91      0.91      0.91      1511



In [ ]:
xgb = XGBClassifier(max_depth=6, n_estimators=1000).fit(train_d2v, train['SECTION']) 
test_pred = xgb.predict(test_d2v) 
test['SECTION'] = test_pred 
submission = test['SECTION'] 
submission.to_excel('xgb_d2v.xlsx', index=False)

In [23]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=30, class_weight='balanced', solver='sag', 
                         multi_class='multinomial', n_jobs=-1, random_state=40, 
                         verbose=1, max_iter = 1000)
clf.fit(xtrain_bow, ytrain)
prediction = clf.predict(xvalid_bow) 
print('accuracy',accuracy_score(yvalid, prediction))  
print('f1score',f1_score(y_true=yvalid, y_pred=prediction, average='weighted'))
print(confusion_matrix(yvalid, prediction))  
print(classification_report(y_true=yvalid, y_pred=prediction))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


max_iter reached after 22 seconds
accuracy 0.9470549305095963
f1score 0.9471211715881157
[[326   6  10   2]
 [  6 518  14  11]
 [  7   7 358   5]
 [  3   6   3 229]]
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       344
           1       0.96      0.94      0.95       549
           2       0.93      0.95      0.94       377
           3       0.93      0.95      0.94       241

   micro avg       0.95      0.95      0.95      1511
   macro avg       0.94      0.95      0.95      1511
weighted avg       0.95      0.95      0.95      1511



C:\Users\Public\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   21.8s finished


In [36]:
clf = LogisticRegression(C=30, class_weight='balanced', solver='sag', 
                         multi_class='multinomial', n_jobs=-1, random_state=40, 
                         verbose=1, max_iter = 1000)
clf.fit(xtrain_w2v, ytrain)
prediction = clf.predict(xvalid_w2v) 
print('accuracy',accuracy_score(yvalid, prediction))  #0.7380050505050505
print('f1score',f1_score(y_true=yvalid, y_pred=prediction, average='weighted'))
print(confusion_matrix(yvalid, prediction))  
print(classification_report(y_true=yvalid, y_pred=prediction))

accuracy 0.9622766379880874
f1score 0.9622771722715061
[[328   5   9   2]
 [  7 524   9   9]
 [  5   4 368   0]
 [  3   4   0 234]]
              precision    recall  f1-score   support

           0       0.96      0.95      0.95       344
           1       0.98      0.95      0.97       549
           2       0.95      0.98      0.96       377
           3       0.96      0.97      0.96       241

   micro avg       0.96      0.96      0.96      1511
   macro avg       0.96      0.96      0.96      1511
weighted avg       0.96      0.96      0.96      1511



In [25]:
from sklearn.naive_bayes import GaussianNB
nb=GaussianNB()
nb.fit(xtrain_bow.toarray(), ytrain)
prediction = clf.predict(xvalid_bow) 
print('accuracy',accuracy_score(yvalid, prediction))  #0.7380050505050505
print('f1score',f1_score(y_true=yvalid, y_pred=prediction, average='weighted'))
print('accuracy',confusion_matrix(yvalid, prediction))  #0.7380050505050505
print('f1score',classification_report(y_true=yvalid, y_pred=prediction))

accuracy 0.9470549305095963
f1score 0.9471211715881157
accuracy [[326   6  10   2]
 [  6 518  14  11]
 [  7   7 358   5]
 [  3   6   3 229]]
f1score               precision    recall  f1-score   support

           0       0.95      0.95      0.95       344
           1       0.96      0.94      0.95       549
           2       0.93      0.95      0.94       377
           3       0.93      0.95      0.94       241

   micro avg       0.95      0.95      0.95      1511
   macro avg       0.94      0.95      0.95      1511
weighted avg       0.95      0.95      0.95      1511



In [34]:
nb=GaussianNB()
nb.fit(xtrain_w2v, ytrain)
prediction = clf.predict(xvalid_w2v) 
print('accuracy',accuracy_score(yvalid, prediction))  #0.7380050505050505
print('f1score',f1_score(y_true=yvalid, y_pred=prediction, average='weighted'))
print('accuracy',confusion_matrix(yvalid, prediction))  #0.7380050505050505
print('f1score',classification_report(y_true=yvalid, y_pred=prediction))

accuracy 0.9622766379880874
f1score 0.9622771722715061
accuracy [[328   5   9   2]
 [  7 524   9   9]
 [  5   4 368   0]
 [  3   4   0 234]]
f1score               precision    recall  f1-score   support

           0       0.96      0.95      0.95       344
           1       0.98      0.95      0.97       549
           2       0.95      0.98      0.96       377
           3       0.96      0.97      0.96       241

   micro avg       0.96      0.96      0.96      1511
   macro avg       0.96      0.96      0.96      1511
weighted avg       0.96      0.96      0.96      1511



In [26]:
from sklearn.linear_model import SGDClassifier
sbg=SGDClassifier(loss='modified_huber',shuffle=True,random_state=101).fit(xtrain_bow, ytrain)
prediction = clf.predict(xvalid_bow) 
print('accuracy',accuracy_score(yvalid, prediction))  #0.7380050505050505
print('f1score',f1_score(y_true=yvalid, y_pred=prediction, average='weighted'))
print('accuracy',confusion_matrix(yvalid, prediction))  #0.7380050505050505
print('f1score',classification_report(y_true=yvalid, y_pred=prediction))

C:\Users\Public\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


accuracy 0.9470549305095963
f1score 0.9471211715881157
accuracy [[326   6  10   2]
 [  6 518  14  11]
 [  7   7 358   5]
 [  3   6   3 229]]
f1score               precision    recall  f1-score   support

           0       0.95      0.95      0.95       344
           1       0.96      0.94      0.95       549
           2       0.93      0.95      0.94       377
           3       0.93      0.95      0.94       241

   micro avg       0.95      0.95      0.95      1511
   macro avg       0.94      0.95      0.95      1511
weighted avg       0.95      0.95      0.95      1511



In [38]:
sbg=SGDClassifier(loss='modified_huber',shuffle=True,random_state=101).fit(xtrain_w2v, ytrain)
prediction = clf.predict(xvalid_w2v) 
print('accuracy',accuracy_score(yvalid, prediction))  #0.7380050505050505
print('f1score',f1_score(y_true=yvalid, y_pred=prediction, average='weighted'))
print('accuracy',confusion_matrix(yvalid, prediction))  #0.7380050505050505
print('f1score',classification_report(y_true=yvalid, y_pred=prediction))

accuracy 0.9622766379880874
f1score 0.9622771722715061
accuracy [[328   5   9   2]
 [  7 524   9   9]
 [  5   4 368   0]
 [  3   4   0 234]]
f1score               precision    recall  f1-score   support

           0       0.96      0.95      0.95       344
           1       0.98      0.95      0.97       549
           2       0.95      0.98      0.96       377
           3       0.96      0.97      0.96       241

   micro avg       0.96      0.96      0.96      1511
   macro avg       0.96      0.96      0.96      1511
weighted avg       0.96      0.96      0.96      1511



In [47]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(learning_rate=0.15, n_estimators=1200,max_depth=9, min_samples_split=1200, min_samples_leaf=60, subsample=0.85, random_state=1, max_features=7,warm_start=True).fit(xtrain_w2v, ytrain)
prediction = gb.predict(xvalid_w2v) 
print('accuracy',accuracy_score(yvalid, prediction))  
print('f1score',f1_score(y_true=yvalid, y_pred=prediction, average='weighted'))
print('accuracy',confusion_matrix(yvalid, prediction))  
print('f1score',classification_report(y_true=yvalid, y_pred=prediction))

accuracy 0.972203838517538
f1score 0.9722131394505193
accuracy [[330   5   6   3]
 [  4 535   5   5]
 [  8   1 368   0]
 [  2   3   0 236]]
f1score               precision    recall  f1-score   support

           0       0.96      0.96      0.96       344
           1       0.98      0.97      0.98       549
           2       0.97      0.98      0.97       377
           3       0.97      0.98      0.97       241

   micro avg       0.97      0.97      0.97      1511
   macro avg       0.97      0.97      0.97      1511
weighted avg       0.97      0.97      0.97      1511



In [49]:
gb = GradientBoostingClassifier(learning_rate=0.15, n_estimators=1200,max_depth=9, min_samples_split=1200, min_samples_leaf=60, subsample=0.85, random_state=1, max_features=7,warm_start=True)
gb.fit(train_w2v, train['SECTION']) 
test_pred = gb.predict(test_w2v) 
test['SECTION'] = test_pred 
submission = test['SECTION']   
submission.to_excel('gb_w2v.xlsx', index=False)   #97.52

In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
dt = DecisionTreeClassifier()
ab= AdaBoostClassifier(n_estimators=1200, base_estimator=dt,random_state=1,learning_rate=0.15).fit(xtrain_w2v, ytrain)
prediction = ab.predict(xvalid_w2v) 
print('accuracy',accuracy_score(yvalid, prediction))  
print('f1score',f1_score(y_true=yvalid, y_pred=prediction, average='weighted'))
print('accuracy',confusion_matrix(yvalid, prediction))  
print('f1score',classification_report(y_true=yvalid, y_pred=prediction))

accuracy 0.9629384513567174
f1score 0.9629301933623033
accuracy [[328  10   4   2]
 [  4 533   8   4]
 [  8   4 365   0]
 [  3   9   0 229]]
f1score               precision    recall  f1-score   support

           0       0.96      0.95      0.95       344
           1       0.96      0.97      0.96       549
           2       0.97      0.97      0.97       377
           3       0.97      0.95      0.96       241

   micro avg       0.96      0.96      0.96      1511
   macro avg       0.96      0.96      0.96      1511
weighted avg       0.96      0.96      0.96      1511



In [ ]:
Public Leaderboard Score-0.9752